In [45]:
# Import libraries
import pandas as pd
from typedb.driver import TypeDB, SessionType, TransactionType
import torch

print("Imports successful.")

# Connect to TypeDB
try:
    with TypeDB.core_driver("localhost:1729") as driver:
        # Access the database manager
        databases = driver.databases

        # Create a new database
        try:
            if not databases.contains("test_db"):
                databases.create("test_db")
                print("Database 'test_db' created successfully.")
            else:
                print("Database 'test_db' already exists.")
        except Exception as e:
            print(f"Error creating database: {e}")

        # Open a session to the new database
        with driver.session("test_db", SessionType.SCHEMA) as session:
            # Start a transaction to define the schema
            with session.transaction(TransactionType.WRITE) as transaction:
                # Define the schema
                schema_query = '''
                define
                person sub entity,
                    owns name;
                name sub attribute,
                    value string;
                '''
                transaction.query.define(schema_query)
                transaction.commit()
                print("Schema defined successfully.")

        # Open a session to the new database
        with driver.session("test_db", SessionType.DATA) as session:
            # Start a transaction to insert data
            with session.transaction(TransactionType.WRITE) as transaction:
                # Insert a simple entity
                insert_query = 'insert $x isa person, has name "John Doe";'
                transaction.query.insert(insert_query)
                transaction.commit()
                print("Inserted entity into 'test_db'.")

            # Start a transaction to read data
            with session.transaction(TransactionType.READ) as transaction:
                match_query = 'match $x isa person, has name $n; get $n;'
                result_iterator = transaction.query.get(match_query)
                for result in result_iterator:
                    print("Retrieved entity with name:", result.get("n").get_value())

        # Delete the test database
        try:
            if databases.contains("test_db"):
                databases.get("test_db").delete()
                print("Database 'test_db' deleted successfully.")
            else:
                print("Database 'test_db' does not exist.")
        except Exception as e:
            print(f"Error deleting database: {e}")

except Exception as e:
    print(f"Error: {e}")


Imports successful.
Database 'test_db' already exists.
Schema defined successfully.
Inserted entity into 'test_db'.
Retrieved entity with name: John Doe
Database 'test_db' deleted successfully.


In [46]:
!pip show typedb-driver

Name: typedb-driver
Version: 2.28.1
Summary: TypeDB Driver for Python
Home-page: https://github.com/vaticle/typedb-driver-python/
Author: Vaticle
Author-email: community@vaticle.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: parse
Required-by: 
